In [1]:
import os, sys
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path

sys.path.append(str(Path.cwd().resolve().parent))

from preprocess import VN30, preprocess_v1
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import MultiTaskLassoCV, LassoCV, Lasso
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_percentage_error, root_mean_squared_error
from pandas import DataFrame

# Bước 1: Tiền xử lý dữ liệu

In [2]:
# Nếu sử dụng LassoCV thì đặt val = 0.0 do CV tự động chia tập train và valid
data = preprocess_v1('ACB', verbose=True)
X_train, y_train = data['train']
X_val, y_val = data['val']
X_test, y_test = data['test']
feature_sc = data['scaler']['feature']
target_sc = data['scaler']['target']

=== Preprocessing ACB ===
Train shape: (1213, 167), Val shape: (0, 167), Test shape: (296, 167)


In [3]:
# for i in range(Y_train.shape[1]):
# 	tscv = TimeSeriesSplit(n_splits=5)
# 	model = LassoCV(cv=tscv)
# 	model.fit(X_train, Y_train[:, i])

# Bước 2: Huấn luyện mô hình
* Lasso (lag = 30, 7 flags)
* Multi-task Lasso (lag = 30, 7 flags)
* Tổng cộng: 14 lần, trên 30 mã, trong mỗi mã chỉ lấy R-quared, in ra lasso.csv 

In [4]:
def train_lasso(X, y):
  tscv = TimeSeriesSplit()
  params = {
    "alpha": np.logspace(-10, 1, 10)
  }
  model = GridSearchCV(Lasso(), param_grid=params, cv=tscv)
  model.fit(X, y)
  return model

In [5]:
def train_multi_task_lasso(X, y):
  tscv = TimeSeriesSplit()
  model = MultiTaskLassoCV(cv=tscv)
  model.fit(X, y)
  return model

# Bước 3: Đánh giá mô hình

In [6]:
model = train_lasso(X_train, y_train)
model

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Lasso(),
             param_grid={'alpha': array([1.00000000e-10, 1.66810054e-09, 2.78255940e-08, 4.64158883e-07,
       7.74263683e-06, 1.29154967e-04, 2.15443469e-03, 3.59381366e-02,
       5.99484250e-01, 1.00000000e+01])})

In [7]:
print(model.best_params_)
print(r2_score(y_test, model.predict(X_test)))

{'alpha': np.float64(7.742636826811277e-06)}
0.9239647258687536


# Đánh giá toàn bộ dữ liệu

In [8]:
flags_list = [
  "none",
  "calendar_feature",
  "rolling_feature",
  "technical_feature",
  "nonlinear_feature",
  "autocorr_feature",
  "trend_feature"
]
flags_dict = {
  flag: False for flag in flags_list if flag != "none"
}
for flag in flags_list:
  print(flag in flags_dict)

False
True
True
True
True
True
True


In [9]:
result = {flag: [] for flag in flags_list}
for code in VN30:
  flags_dict = {
    flag: False for flag in flags_list if flag != "none"
  }
  for flag in flags_list:
    if flag in flags_dict:
      flags_dict[flag] = True
    data = preprocess_v1(code, **flags_dict)
    X_train, y_train = data['train']
    X_val, y_val = data['val']
    X_test, y_test = data['test']
    feature_sc = data['scaler']['feature']
    target_sc = data['scaler']['target']
    model = train_lasso(X_train, y_train)
    y_test_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_test_pred)
    result[flag].append(r2)
    print(code, flags_dict, r2)

df = DataFrame(result, index=VN30)
df.to_csv("lasso.csv")

ACB {'calendar_feature': False, 'rolling_feature': False, 'technical_feature': False, 'nonlinear_feature': False, 'autocorr_feature': False, 'trend_feature': False} 0.9340326573579159
ACB {'calendar_feature': True, 'rolling_feature': False, 'technical_feature': False, 'nonlinear_feature': False, 'autocorr_feature': False, 'trend_feature': False} 0.9343025909368063
ACB {'calendar_feature': True, 'rolling_feature': True, 'technical_feature': False, 'nonlinear_feature': False, 'autocorr_feature': False, 'trend_feature': False} 0.9357460783620701
ACB {'calendar_feature': True, 'rolling_feature': True, 'technical_feature': True, 'nonlinear_feature': False, 'autocorr_feature': False, 'trend_feature': False} 0.9379046052420473
ACB {'calendar_feature': True, 'rolling_feature': True, 'technical_feature': True, 'nonlinear_feature': True, 'autocorr_feature': False, 'trend_feature': False} 0.9250827886181284
ACB {'calendar_feature': True, 'rolling_feature': True, 'technical_feature': True, 'nonlin

In [10]:
result = {flag: [] for flag in flags_list}
for code in VN30:
  flags_dict = {
    flag: False for flag in flags_list if flag != "none"
  }
  for flag in flags_list:
    if flag in flags_dict:
      flags_dict[flag] = True
    data = preprocess_v1(code, **flags_dict)
    X_train, y_train = data['train']
    X_val, y_val = data['val']
    X_test, y_test = data['test']
    feature_sc = data['scaler']['feature']
    target_sc = data['scaler']['target']
    model = train_multi_task_lasso(X_train, y_train)
    y_test_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_test_pred)
    result[flag].append(r2)
    print(code, flags_dict, r2)

df = DataFrame(result, index=VN30)
df.to_csv("multi-task-lasso.csv")

ACB {'calendar_feature': False, 'rolling_feature': False, 'technical_feature': False, 'nonlinear_feature': False, 'autocorr_feature': False, 'trend_feature': False} 0.9356495647459988
ACB {'calendar_feature': True, 'rolling_feature': False, 'technical_feature': False, 'nonlinear_feature': False, 'autocorr_feature': False, 'trend_feature': False} 0.9358645303191608
ACB {'calendar_feature': True, 'rolling_feature': True, 'technical_feature': False, 'nonlinear_feature': False, 'autocorr_feature': False, 'trend_feature': False} 0.937409801400511
ACB {'calendar_feature': True, 'rolling_feature': True, 'technical_feature': True, 'nonlinear_feature': False, 'autocorr_feature': False, 'trend_feature': False} 0.9368275907816046
ACB {'calendar_feature': True, 'rolling_feature': True, 'technical_feature': True, 'nonlinear_feature': True, 'autocorr_feature': False, 'trend_feature': False} 0.9323952793286687
ACB {'calendar_feature': True, 'rolling_feature': True, 'technical_feature': True, 'nonline